In [1]:
!pip install torch transformers datasets accelerate bitsandbytes peft trl
!pip install hf_transfer
!pip install "unsloth[full]" --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 8.4 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 6.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.9 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.8 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.3 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 14.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 22.8 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26/26 [trl]32m25/26 [trl]sets]e]s]ub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 12.0 MB/s  0:00:00
   ━━━━━━━━━━━

In [2]:
from datasets import load_dataset

# 1. Define the dataset ID
dataset_id = "ruslanmv/ai-medical-chatbot"

# 2. Load the dataset
# We'll just load the 'train' split for now.
try:
    dataset = load_dataset(dataset_id, split="train")

    # 3. Print the first example to see its structure
    print("--- Example from the dataset: ---")
    print(dataset[0])

    print(f"\nSuccessfully loaded {len(dataset)} examples.")

except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    print(f"Please check the dataset ID '{dataset_id}' and your internet connection.")

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

--- Example from the dataset: ---
{'Description': 'Q. What does abutment of the nerve root mean?', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?', 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}

Successfully loaded 256916 examples.


In [3]:
from datasets import load_dataset

# --- We can combine Step 1 and 2 ---

# 1. Define the dataset ID and load it
dataset_id = "ruslanmv/ai-medical-chatbot"
try:
    dataset = load_dataset(dataset_id, split="train")
except Exception as e:
    print(f"Failed to load dataset: {e}")
    exit() # Exit if dataset loading fails

# 2. Define the formatting function
def format_mistral_prompt(sample):
    """
    This function takes a sample (a dictionary) from the dataset
    and formats it into the Mistral-Instruct prompt template.

    We map 'Patient' to the instruction and 'Doctor' to the response.
    """
    patient_query = sample.get("Patient", "")
    doctor_response = sample.get("Doctor", "")

    # Create the formatted prompt
    prompt = f"<s>[INST] {patient_query} [/INST] {doctor_response} </s>"

    # The SFTTrainer expects a column named "text" by default
    return {"text": prompt}

# 3. Apply the formatting to the entire dataset
# .map() will process every example and add the new "text" column
print("\nStarting dataset formatting...")
formatted_dataset = dataset.map(format_mistral_prompt)
print("Formatting complete.")

# 4. (Optional) Check your work!
print("\n--- EXAMPLE OF FORMATTED DATA: ---")
print(formatted_dataset[0]['text'])


Starting dataset formatting...


Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

Formatting complete.

--- EXAMPLE OF FORMATTED DATA: ---
<s>[INST] Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for annular bulging and tear? [/INST] Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> </s>


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


In [11]:
# === 1. INSTALL LIBRARIES ===
# We use the recommended Unsloth install and -q for a quiet installation
!pip install "unsloth[colab-new]" -q
!pip install hf_transfer -q

print("✅ Libraries installed.")

# === 2. IMPORTS (UNSLOTH FIRST!) ===
# Unsloth MUST be imported before transformers, trl, or peft
from unsloth import FastMistralModel
import torch

# Now import everything else
from transformers import AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig

print("🦥 Unsloth and all libraries imported correctly.")

# === 3. DATASET LOADING & FORMATTING ===
dataset_id = "ruslanmv/ai-medical-chatbot"
try:
    dataset = load_dataset(dataset_id, split="train")
except Exception as e:
    print(f"Failed to load dataset: {e}")
    exit()

def format_mistral_prompt(sample):
    patient_query = sample.get("Patient", "")
    doctor_response = sample.get("Doctor", "")
    # Format according to Mistral-Instruct template
    prompt = f"<s>[INST] {patient_query} [/INST] {doctor_response} </s>"
    return {"text": prompt}

print("\nStarting dataset formatting...")
formatted_dataset = dataset.map(format_mistral_prompt)
print("Formatting complete.")
print("\n--- EXAMPLE OF FORMATTED DATA: ---")
print(formatted_dataset[0]['text'])


# === 4. MODEL & TOKENIZER LOADING (Corrected) ===
# This is the first part of the fix: Load the model WITHOUT LoRA args
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
max_seq_length = 2048 

print(f"\nLoading base model with Unsloth: {model_id}")
model, tokenizer = FastMistralModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = None,       # (None lets Unsloth choose bfloat16)
    load_in_4bit = True,
    device_map = "auto",
)
print("✅ Base model loaded successfully.")

# --- Configure Model & Tokenizer ---
model.config.use_cache = False # Necessary for training
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# === 5. APPLY PEFT (LORA) CONFIG (Corrected) ===
# This is the second part of the fix:
# We use Unsloth's specific function `get_peft_model` to apply adapters
print("\nApplying LoRA adapters with Unsloth...")
model = FastMistralModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)
print("✅ Unsloth PEFT adapters applied successfully.")


# === 6. TRAINER CONFIG ===
sft_config = SFTConfig(
    output_dir = "./mistral-medical-adapter",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    optim = "paged_adamw_8bit",
    num_train_epochs = 1,
    learning_rate = 2e-4,
    lr_scheduler_type = "linear",
    bf16 = True,  # bf16 is supported and recommended by Unsloth
    logging_steps = 25,
    save_strategy = "epoch",
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    report_to = "none", # Disables wandb
)

trainer = SFTTrainer(
    model = model,
    train_dataset = formatted_dataset,
    args = sft_config,
    tokenizer = tokenizer,
)
print("SFTTrainer initialized.")


# === 7. START TRAINING ===
print("\n--- 🚀 Starting Training... ---")
trainer.train()
print("--- ✅ Training Complete! ---")

# === 8. SAVE THE FINAL ADAPTER ===
final_adapter_path = "./mistral-medical-adapter-final"
trainer.model.save_pretrained(final_adapter_path)
tokenizer.save_pretrained(final_adapter_path) # Also save the tokenizer
print(f"✅ Final adapter and tokenizer saved to {final_adapter_path}")

✅ Libraries installed.
🦥 Unsloth and all libraries imported correctly.

Starting dataset formatting...
Formatting complete.

--- EXAMPLE OF FORMATTED DATA: ---
<s>[INST] Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for annular bulging and tear? [/INST] Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> </s>

Loading base model with Unsloth: mistralai/Mistral-7B-Instruct-v0.2
==((====))==  Unsloth 2025.11.2: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.526 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

mistralai/Mistral-7B-Instruct-v0.2 does not have a padding token! Will use pad_token = <unk>.
✅ Base model loaded successfully.

Applying LoRA adapters with Unsloth...


Unsloth 2025.11.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Unsloth PEFT adapters applied successfully.


...

SFTTrainer initialized.

--- 🚀 Starting Training... ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 256,916 | Num Epochs = 1 | Total steps = 16,058
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


KeyboardInterrupt: 

In [6]:
# === 5. TRAINER CONFIG ===
sft_config = SFTConfig(
    output_dir="./mistral-medical-adapter",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    bf16=True,  # bf16 is supported and recommended by Unsloth
    logging_steps=25,
    save_strategy="epoch",
    dataset_text_field="text",
    max_seq_length=max_seq_length, # Pass max_seq_length
    report_to="none", # Disables wandb
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=sft_config,
    tokenizer=tokenizer, # Pass the tokenizer
)
print("SFTTrainer initialized.")

Unsloth: Tokenizing ["text"] (num_proc=36):   0%|          | 0/256916 [00:00<?, ? examples/s]

SFTTrainer initialized.


In [ ]:
# Save base model + LoRA adapter together
model.save_pretrained("./mistral-full-model")
tokenizer.save_pretrained("./mistral-full-model")


In [7]:
print("\n--- 🚀 Starting Training... ---")

# 3. Start the Training!
# ---
trainer.train()

print("--- ✅ Training Complete! ---")

# 4. Save the Final Adapter
# ---
final_adapter_path = "./mistral-medical-adapter-final"
trainer.model.save_pretrained(final_adapter_path)
print(f"✅ Final adapter saved to {final_adapter_path}")

The model is already on multiple devices. Skipping the move to device specified in `args`.



--- 🚀 Starting Training... ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 256,916 | Num Epochs = 1 | Total steps = 16,058
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn